In [3]:
import pandas as pd


In [8]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [9]:
ingest_create_athena_db_passed = False

## Import PyAthena

In [10]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

## Create Athena Database

In [11]:
database_name = "compens"

In [12]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [14]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS compens


In [15]:
pd.read_sql(statement, conn)

""


## Verify The Database Has Been Created Succesfully

In [16]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,compens
1,default
2,dsoaws


In [156]:
#Directory for the database
compens_dir = 's3://508-team4/data'

In [26]:
test = pd.read_csv('s3://508-team4/data/la_compensation.csv')
test.info()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753548 entries, 0 to 753547
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   record_nbr                     753548 non-null  object 
 1   pay_year                       753548 non-null  int64  
 2   department_no                  753548 non-null  int64  
 3   department_title               753548 non-null  object 
 4   job_class_pgrade               753004 non-null  object 
 5   job_title                      753004 non-null  object 
 6   employment_type                753548 non-null  object 
 7   job_status                     753548 non-null  object 
 8   mou                            752862 non-null  object 
 9   mou_title                      752755 non-null  object 
 10  regular_pay                    753548 non-null  float64
 11  overtime_pay                   753114 non-null  float64
 12  all_other_pay                 

In [40]:
table_name_test ='la_compens'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name_test}', conn)

create_table_test = f""" 
CREATE TABLE IF NOT EXISTS {database_name}.{table_name_test} AS 
SELECT 
record_nbr,
CAST(pay_year AS INT) AS year,
department_no, 
CAST(department_title AS VARCHAR) AS department, 
job_class_pgrade, 
job_title, 
employment_type, 
job_status, 
mou, 
mou_title,
CAST(regular_pay AS FLOAT) AS base_salary,
CAST(overtime_pay AS FLOAT) AS overtime,
CAST(all_other_pay AS FLOAT) AS irregular_cash,
CAST(total_pay AS FLOAT) AS total_cash,
CAST(city_retirement_contributions AS FLOAT) AS retirement,
CAST(benefit_pay AS FLOAT) AS health,
gender,
ethnicity,
retirement + health AS total_benefits,
total_cash + retirement + health AS total_compensation 
FROM compens.la_compensation
"""

pd.read_sql(create_table_test, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name_test} LIMIT 5', conn)



DatabaseError: Execution failed on sql:  
CREATE TABLE IF NOT EXISTS compens.la_compens AS 
SELECT 
record_nbr,
CAST(pay_year AS INT) AS year,
department_no, 
CAST(department_title AS VARCHAR) AS department, 
job_class_pgrade, 
job_title, 
employment_type, 
job_status, 
mou, 
mou_title,
CAST(regular_pay AS FLOAT) AS base_salary,
CAST(overtime_pay AS FLOAT) AS overtime,
CAST(all_other_pay AS FLOAT) AS irregular_cash,
CAST(total_pay AS FLOAT) AS total_cash,
CAST(city_retirement_contributions AS FLOAT) AS retirement,
CAST(benefit_pay AS FLOAT) AS health,
gender,
ethnicity,
retirement + health AS total_benefits,
total_cash + retirement + health AS total_compensation 
FROM compens.la_compensation

SYNTAX_ERROR: line 15:3: Unknown type: float. You may need to manually clean the data at location 's3://sagemaker-us-east-1-924490614652/athena/staging/tables/7ed6d3f2-f39f-4a16-8a89-433b4c453dfe' before retrying. Athena will not delete data in your account.
unable to rollback

In [43]:
df_show = pd.read_sql('SHOW TABLES IN compens', conn)
df_show.head(5)


,tab_name
0,la
1,la_compensation


In [167]:
table_name1 ='la_compensation'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name1}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name1}(
                record_nbr string,
                pay_year int,
                department_no string,
                department_title string,
                job_class_pgrade int,
                job_title string,
                employment_type string,
                job_status string,
                mou string,
                mou_title string,
                regular_pay float,
                overtime_pay float,
                all_other_pay float,
                total_pay float,
                city_retirement_contributions float,
                benefit_pay float,
                gender string,
                ethnicity string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{compens_dir}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)
                

,record_nbr,pay_year,department_no,department_title,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,regular_pay,overtime_pay,all_other_pay,total_pay,city_retirement_contributions,benefit_pay,gender,ethnicity
0,303030303632,2017,98,WATER AND POWER,None,CUSTODIAN,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,55725.24,4785.05,2021.84,62532.13,3678.0,23508.9,FEMALE,HISPANIC
1,3030303036,2017,98,WATER AND POWER,None,UTILITY ADMINISTRATOR,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,139174.88,16340.50,6170.49,161685.88,9186.0,23508.9,FEMALE,ASIAN AMERICAN
2,303030313232,2017,98,WATER AND POWER,None,WATER SERVICES MANAGER,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,245879.12,0.00,12504.30,258383.42,16228.0,23508.9,MALE,BLACK
3,303030313632,2017,98,WATER AND POWER,None,WTR TRTMT OPR,FULL_TIME,ACTIVE,6,STEAM PLANT AND WATER SUPPLY UNIT,101494.34,7824.99,12630.52,121949.85,6699.0,23508.9,MALE,ASIAN AMERICAN
4,303030323632,2017,98,WATER AND POWER,None,ELTL MCHC,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,101345.12,22284.37,1566.75,125196.24,6689.0,23508.9,MALE,HISPANIC


In [143]:
# change_table = f'''ALTER TABLE {database_name}.{table_name1} CHANGE (pay_year year int, department_title department string)'''
# cursor.execute(change_table)
# conn.commit()
# # pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)

In [124]:
cursor = conn.cursor()

cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE pay_year year int')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE department_title department string')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE regular_pay base_salary float')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE overtime_pay overtime float')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE all_other_pay irregular_cash float')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE total_pay total_cash float')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE city_retirement_contributions retirement float')
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} CHANGE benefit_pay health float')

conn.commit()
  
pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)
   

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,base_salary,overtime,irregular_cash,total_cash,retirement,health,gender,ethnicity
0,303030303632,2017,98,WATER AND POWER,None,CUSTODIAN,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,55725.24,4785.05,2021.84,62532.13,3678.0,23508.9,FEMALE,HISPANIC
1,3030303036,2017,98,WATER AND POWER,None,UTILITY ADMINISTRATOR,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,139174.88,16340.50,6170.49,161685.88,9186.0,23508.9,FEMALE,ASIAN AMERICAN
2,303030313232,2017,98,WATER AND POWER,None,WATER SERVICES MANAGER,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,245879.12,0.00,12504.30,258383.42,16228.0,23508.9,MALE,BLACK
3,303030313632,2017,98,WATER AND POWER,None,WTR TRTMT OPR,FULL_TIME,ACTIVE,6,STEAM PLANT AND WATER SUPPLY UNIT,101494.34,7824.99,12630.52,121949.85,6699.0,23508.9,MALE,ASIAN AMERICAN
4,303030323632,2017,98,WATER AND POWER,None,ELTL MCHC,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,101345.12,22284.37,1566.75,125196.24,6689.0,23508.9,MALE,HISPANIC


In [ ]:
SELECT column1, column2, column3, column1 + column2 + column3 AS sum_column
FROM your_table;

In [147]:
cursor = conn.cursor()
cursor.execute(f'ALTER TABLE {database_name}.{table_name1} ADD (city_id INT DEFAULT 1)')
#cursor.execute(f'UPDATE {database_name}.{table_name1} SET city_id = 1')
#ALTER TABLE compens.la_compensation ADD COLUMN new_column INT DEFAULT 1;

conn.commit()
  
pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)

Failed to execute query.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyathena/common.py", line 305, in _execute
    **request
  File "/opt/conda/lib/python3.7/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 379, in __call__
    do = self.iter(retry_state=retry_state)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 314, in iter
    return fut.result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 382, in __call__
    result = fn(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 530, in _api_call
    return

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:41: mismatched input '('. Expecting: '.', 'ADD'

In [39]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}